# Notebook for initial data exploration

In [ ]:
#import necessary libraries for data analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Exploration of addresses.csv

### Identify duplicated supplier IDs

In [ ]:
addresses = pd.read_csv('../data/raw/addresses.csv', header=0)

# Find duplicate key-value pairs: supplier_id with multiple countries
duplicates = addresses.groupby('supplier_id')['country'].nunique().reset_index()
duplicates = duplicates[duplicates['country'] > 1]

# Show affected supplier_ids and their countries
for id in duplicates['supplier_id']:
    countries = addresses[addresses['supplier_id'] == id]['country'].unique()
    indices = addresses[addresses['supplier_id'] == id].index
    print(f"Supplier ID {id} is present at multiple positions {indices.values}\n")
    print(addresses.loc[indices])

### Identify switched cases in addresses.csv

In [ ]:
# Identify country names with inconsistent capitalization
country_variants = addresses.groupby(addresses['country'].str.lower())['country'].unique()
for base, variants in country_variants.items():
    if len(variants) > 1:
        print(f"Country '{base}' has multiple variants: {variants}")

## Exploration of articles.csv

### Identify switched cases in articles.csv

In [ ]:
articles = pd.read_csv('../data/raw/articles.csv', header=0)

# Identify Article names with inconsistent capitalization
article_variants = articles.groupby(articles['Industry'].str.lower())['Industry'].unique()
for base, variants in article_variants.items():
    if len(variants) > 1:
        print(f"Industry '{base}' has multiple variants: {variants}")

## Exploration of indices.csv

### Checking for duplicates

In [ ]:
indices = pd.read_csv('../data/raw/indices.csv', header=0)

# Find duplicated country_id values
duplicate_ids = indices[indices.duplicated('country_id', keep=False)]

# Show all rows with duplicated country_id
if not duplicate_ids.empty:
    print("Duplicated country_id entries:")
    print(duplicate_ids)
else:
    print("No duplicated country_id found.")

### Identify missing values for human_rights_index and enivronmental_risk

In [ ]:
indices = pd.read_csv('../data/raw/indices.csv', header=0)


# Convert both columns to numeric, non-convertible values become NaN
indices[['human_rights_index', 'enivronmental_risk']] = indices[['human_rights_index', 'enivronmental_risk']].apply(pd.to_numeric, errors='coerce')
# Count missing values for hri
missing_count_hri = indices['human_rights_index'].isna().sum()
present_values_count_hri = indices['human_rights_index'].notna().sum()

missing_count_env = indices['enivronmental_risk'].isna().sum()
present_values_count_env = indices['enivronmental_risk'].notna().sum()

print(f"Number of present values in 'human_rights_index': {present_values_count_hri}")
print(f"Number of missing values in 'human_rights_index': {missing_count_hri}")

print(f"Number of present values in 'enivronmental_risk': {present_values_count_env}")
print(f"Number of missing values in 'enivronmental_risk': {missing_count_env}")


### Plotting human rights index values

In [ ]:
# Plot a histogram of 'human_rights_index' with a separate bar for missing values

# Plot histogram for available values
plt.figure(figsize=(10,6))
plt.hist(indices['human_rights_index'].dropna(), bins=100, range=(0,100), color='skyblue', edgecolor='black', label='Available values')

# Add a bar for missing values
plt.bar(105, missing_count_hri, width=2, color='orange', label='Missing values')

plt.xlabel('Human Rights Index')
plt.ylabel('Count')
plt.title('Distribution of Human Rights Index (including missing values)')
plt.legend()
plt.xticks(list(np.linspace(0,100,11)) + [110], labels=[str(int(x)) for x in np.linspace(0,100,11)] + ['Missing'])
plt.show()

### Plotting environmental risk values

In [ ]:
# Plot a histogram of 'enivronmental_risk' with a separate bar for missing values

# Plot histogram for available values
plt.figure(figsize=(10,6))
plt.hist(indices['enivronmental_risk'].dropna(), bins=100, range=(0,100), color='green', edgecolor='black', label='Available values')

# Add a bar for missing values
plt.bar(105, missing_count_env, width=2, color='orange', label='Missing values')

plt.xlabel('Human Rights Index')
plt.ylabel('Count')
plt.title('Distribution of Human Rights Index (including missing values)')
plt.legend()
plt.xticks(list(np.linspace(0,100,11)) + [110], labels=[str(int(x)) for x in np.linspace(0,100,11)] + ['Missing'])
plt.show()

### Plotting distribution of environmental risk values

In [ ]:
# Plot the percentage distribution of all non-null values for 'enivronmental_risk'
env_risk_counts = indices['enivronmental_risk'].dropna().value_counts(normalize=True).sort_index() * 100
plt.figure(figsize=(12,6))
env_risk_counts.plot(kind='bar', color='green', edgecolor='black')
plt.xlabel('Environmental Risk')
plt.ylabel('Percentage (%)')
plt.title('Percentage Distribution of Environmental Risk (non-null values)')
plt.show()

### Investigating correlation between hri and env

In [ ]:
# Scatterplot: human_rights_index vs. enivronmental_risk with point size by frequency
counts = indices.groupby(['human_rights_index', 'enivronmental_risk']).size().reset_index(name='count')
plt.figure(figsize=(8,6))
plt.scatter(counts['human_rights_index'], counts['enivronmental_risk'], s=counts['count']*10, alpha=0.7, color='purple', edgecolor='k')
plt.xlabel('Human Rights Index')
plt.ylabel('Environmental Risk')
plt.title('Scatterplot of Human Rights Index vs. Environmental Risk (point size = frequency)')
plt.grid(True)
plt.show()

## Exploration of suppliers.csv

### Checking for duplicates

In [ ]:
suppliers = pd.read_csv('../data/raw/suppliers.csv', header=0)

# Find duplicated supplier_id values
duplicate_ids = suppliers[suppliers.duplicated('supplier_id', keep=False)]

# Show all rows with duplicated supplier_id
if not duplicate_ids.empty:
    print("Duplicated supplier_id entries:")
    print(duplicate_ids)
else:
    print("No duplicated supplier_id found.")

### Investigating amount of active suppliers with and without a valid certificate

In [ ]:
# Calculate the share of unique suppliers with status 'active' among all unique suppliers
suppliers = pd.read_csv('../data/raw/suppliers.csv', header=0)

# Remove duplicate supplier_id entries
unique_suppliers = suppliers.drop_duplicates(subset='supplier_id')

# Calculate the number of active suppliers
active_count = (unique_suppliers['status'] == 'active').sum()

# Calculate the number of suppliers with a valid certificate
certificate_count = (unique_suppliers['certificates_valid'] == 'yes').sum()

# Calculate the number of active suppliers with a valid certificate
active_certificate_count = ((unique_suppliers['status'] == 'active') & (unique_suppliers['certificates_valid'] == 'yes')).sum()

active_no_certificate_count = active_count - active_certificate_count

total_count = len(unique_suppliers)
active_share = active_count / total_count if total_count > 0 else np.nan
certificate_share = certificate_count / total_count if total_count > 0 else np.nan
active_certificate_share = active_certificate_count / total_count if total_count > 0 else np.nan
active_no_certificate_share = active_no_certificate_count / total_count if total_count > 0 else np.nan

print(f"Share of active suppliers: {active_share:.2%} ({active_count} of {total_count})")
print(f"Share of suppliers with a valid certificate: {certificate_share:.2%} ({certificate_count} of {total_count})")
print(f"Share of active suppliers with a valid certificate: {active_certificate_share:.2%} ({active_certificate_count} of {total_count})")
print(f"Share of active suppliers without a valid certificate: {active_no_certificate_share:.2%} ({active_no_certificate_count} of {total_count})")

### Plotting revenue of suppliers to identfiy outliers

In [ ]:
# Make suppliers with revenue=0 visible by adding 1 to all revenues (log scale)
unique_suppliers_sorted = unique_suppliers.sort_values('supplier_id').reset_index(drop=True)
unique_suppliers_sorted['revenue_plus1'] = unique_suppliers_sorted['total_company_revenue'] + 1

combinations = [
    ('active', 'yes'),
    ('active', 'no'),
    ('inactive', 'yes'),
    ('inactive', 'no')
 ]
color_map = {'active': 'green', 'inactive': 'red'}
marker_map = {'yes': 'o', 'no': 'x'}
labels = {
    ('active', 'yes'): 'Active & Valid Certificate',
    ('active', 'no'): 'Active & No Certificate',
    ('inactive', 'yes'): 'Inactive & Valid Certificate',
    ('inactive', 'no'): 'Inactive & No Certificate'
}

plt.figure(figsize=(12,6))
for status, cert in combinations:
    subset = unique_suppliers_sorted[(unique_suppliers_sorted['status'] == status) & (unique_suppliers_sorted['certificates_valid'] == cert)]
    plt.scatter(subset.index, subset['revenue_plus1'],
                c=color_map[status], marker=marker_map[cert],
                label=labels[(status, cert)], alpha=0.7, edgecolor='k')

plt.xlabel('Supplier (sorted by supplier_id)')
plt.ylabel('Total Company Revenue (+1)')
plt.title('Supplier Revenue by Status and Certificate Validity (log scale, 0 values visible)')
plt.legend(loc='upper right')
plt.yscale('log')
plt.show()

## Exploration of orders.csv

### Checking for duplicates

In [ ]:
orders = pd.read_csv('../data/raw/orders.csv', header=0)

# Find duplicated order_id values
duplicate_ids = orders[orders.duplicated('order_id', keep=False)]

# Show all rows with duplicated order_id
if not duplicate_ids.empty:
    print("Duplicated order_id entries:")
    print(duplicate_ids)
else:
    print("No duplicated order_id found.")

### Plotting orders on quantity and price per quantity

In [ ]:
# Read orders
orders = pd.read_csv('../data/raw/orders.csv', header=0)

# Read articles and clean Article_IDs
articles = pd.read_csv('../data/raw/articles.csv', header=0)
articles['Article_ID_clean'] = articles['Article_ID'].astype(str).str.replace('"', '').astype(int)

# Also clean article_id in orders (if necessary)
orders['article_id_clean'] = orders['article_id'].astype(int)

# Merge: Add article name to orders
orders_merged = orders.merge(articles, left_on='article_id_clean', right_on='Article_ID_clean', how='left')

# Assign colors to articles
import itertools
colors = plt.get_cmap('tab10', len(orders_merged['Article'].unique()))
article_names = orders_merged['Article'].unique()
color_map = {name: colors(i) for i, name in enumerate(article_names)}

plt.figure(figsize=(12,8))
for i, article in enumerate(article_names):
    subset = orders_merged[orders_merged['Article'] == article]
    plt.scatter(subset['quantity'], subset['price_per_quantity'],
                color=color_map[article], label=article, alpha=0.7, edgecolor='k')

plt.xlabel('Quantity')
plt.ylabel('Price per Quantity')
plt.title('Scatterplot: Quantity vs. Price per Quantity by Article')
plt.legend(title='Article Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Plotting order associated industries on quantity and price per quantity

In [ ]:
# Scatterplot: Group articles by industry (case-insensitive)
orders = pd.read_csv('../data/raw/orders.csv', header=0)
articles = pd.read_csv('../data/raw/articles.csv', header=0)
articles['Article_ID_clean'] = articles['Article_ID'].astype(str).str.replace('"', '').astype(int)
articles['Industry_clean'] = articles['Industry'].str.lower()

orders['article_id_clean'] = orders['article_id'].astype(int)
orders_merged = orders.merge(articles, left_on='article_id_clean', right_on='Article_ID_clean', how='left')

# Get unique industries (case-insensitive)
industry_names = articles['Industry_clean'].unique()
colors = plt.get_cmap('Set1', len(industry_names))
color_map = {name: colors(i) for i, name in enumerate(industry_names)}

plt.figure(figsize=(12,8))
for i, industry in enumerate(industry_names):
    subset = orders_merged[orders_merged['Industry_clean'] == industry]
    plt.scatter(subset['quantity'], subset['price_per_quantity'],
                color=color_map[industry], label=industry.capitalize(), alpha=0.7, edgecolor='k')

plt.xlabel('Quantity')
plt.ylabel('Price per Quantity')
plt.title('Scatterplot: Quantity vs. Price per Quantity by Industry')
plt.legend(title='Industry', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Plotting suppliers and the industries they belong to by their total order value

In [ ]:
# Bar plot: Total order value per supplier, colored by industry (red = multiple industries)
orders = pd.read_csv('../data/raw/orders.csv', header=0)
articles = pd.read_csv('../data/raw/articles.csv', header=0)
orders['supplier_id_clean'] = orders['supplier_id'].astype(str).str.replace('"', '').astype(int)
articles['Article_ID_clean'] = articles['Article_ID'].astype(str).str.replace('"', '').astype(int)
orders['article_id_clean'] = orders['article_id'].astype(int)
orders_merged = orders.merge(articles, left_on='article_id_clean', right_on='Article_ID_clean', how='left')
orders_merged['Industry_clean'] = orders_merged['Industry'].str.lower()
supplier_industries = orders_merged.groupby('supplier_id_clean')['Industry_clean'].agg(lambda x: set(x.dropna()))
def get_color(industries):
    if len(industries) > 1:
        return 'red'
    industry = list(industries)[0] if industries else 'unknown'
    color_map = {
        'metals': 'steelblue',
        'electronics': 'orange',
        'plastics': 'green'
    }
    return color_map.get(industry, 'gray')
supplier_colors = supplier_industries.apply(get_color)
supplier_order_value = orders_merged.groupby('supplier_id_clean')['order_value'].sum().sort_values(ascending=False)
bar_colors = supplier_colors.reindex(supplier_order_value.index).tolist()
import matplotlib.pyplot as plt
plt.figure(figsize=(14,6))
supplier_order_value.plot(kind='bar', color=bar_colors, edgecolor='black')
plt.xlabel('Supplier ID')
plt.ylabel('Total Order Value')
plt.title('Total Order Value per Supplier (colored by Industry, red = multiple)')
import matplotlib.patches as mpatches
legend_patches = [
    mpatches.Patch(color='steelblue', label='Metals'),
    mpatches.Patch(color='orange', label='Electronics'),
    mpatches.Patch(color='green', label='Plastics'),
    mpatches.Patch(color='red', label='Multiple Industries'),
    mpatches.Patch(color='gray', label='Unknown')
]
plt.legend(handles=legend_patches, title='Industry')
plt.tight_layout()
plt.show()

### Plotting suppliers and the industries they belong to by their number of orders

In [ ]:
# Bar plot: Number of orders per supplier, colored by industry (red = multiple industries)

orders = pd.read_csv('../data/raw/orders.csv', header=0)
articles = pd.read_csv('../data/raw/articles.csv', header=0)
orders['supplier_id_clean'] = orders['supplier_id'].astype(str).str.replace('"', '').astype(int)
articles['Article_ID_clean'] = articles['Article_ID'].astype(str).str.replace('"', '').astype(int)
orders['article_id_clean'] = orders['article_id'].astype(int)
orders_merged = orders.merge(articles, left_on='article_id_clean', right_on='Article_ID_clean', how='left')
orders_merged['Industry_clean'] = orders_merged['Industry'].str.lower()
supplier_industries = orders_merged.groupby('supplier_id_clean')['Industry_clean'].agg(lambda x: set(x.dropna()))
def get_color(industries):
    if len(industries) > 1:
        return 'red'
    industry = list(industries)[0] if industries else 'unknown'
    color_map = {
        'metals': 'steelblue',
        'electronics': 'orange',
        'plastics': 'green'
    }
    return color_map.get(industry, 'gray')
supplier_colors = supplier_industries.apply(get_color)
supplier_order_counts = orders_merged.groupby('supplier_id_clean')['order_id'].count().sort_values(ascending=False)
bar_colors = supplier_colors.reindex(supplier_order_counts.index).tolist()
import matplotlib.pyplot as plt
plt.figure(figsize=(14,6))
supplier_order_counts.plot(kind='bar', color=bar_colors, edgecolor='black')
plt.xlabel('Supplier ID')
plt.ylabel('Number of Orders')
plt.title('Number of Orders per Supplier (colored by Industry, red = multiple)')
import matplotlib.patches as mpatches
legend_patches = [
    mpatches.Patch(color='steelblue', label='Metals'),
    mpatches.Patch(color='orange', label='Electronics'),
    mpatches.Patch(color='green', label='Plastics'),
    mpatches.Patch(color='red', label='Multiple Industries'),
    mpatches.Patch(color='gray', label='Unknown')
]
plt.legend(handles=legend_patches, title='Industry')
plt.tight_layout()
plt.show()